# **Parameter Efficient Fine Tuning using LoRA adapter**

Paper being referred: "*LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS*" (https://arxiv.org/pdf/2106.09685.pdf)

In [4]:
# Mount into drive

from google.colab import drive

drive.mount("/content/drive")

# Change directory to the package folder
#!! This should be the base directory of the project, this directory should look like the screenshot below.

%cd /content/drive/MyDrive/llm_finetuning-main/notebooks

# Verify the contents of the current folder
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/llm_finetuning-main/notebooks
context_distillation_mnli.ipynb		  results
few_shot_context_distillation_mnli.ipynb  results1
pbft_cola_baseline.ipynb		  vanilla_cola_adaptive_v2.ipynb
pbft_mnli_baseline_hansOOD_350m.ipynb	  vanilla_cola_baseline_350M.ipynb
pbft_mnli_baseline_hansOOD.ipynb	  vanilla_cola_baseline.ipynb
pbft_mnli_baseline.ipynb		  vanilla_cola_baseline_v2.ipynb
peft_LoRA_on_cola.ipynb			  vanilla_mnli_baseline_Hans_350M.ipynb
peft_LoRA_on_RTE.ipynb			  vanilla_mnli_baseline_Hans.ipynb
peft_LoRA_on_RTE_v3.ipynb		  wandb
pre_trained_opt_with_inference.ipynb


In [5]:
!pip install -q transformers accelerate bitsandbytes datasets torch peft tqdm

In [6]:
# Load required libraries

from datasets import load_dataset, ClassLabel
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AdamW, AutoConfig
from peft import LoraConfig, get_peft_model
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

# Set seed, load RTE dataset

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

rte_data = load_dataset("glue", "rte") # load rte data
hans_data = load_dataset("hans")

# preprocess RTE dataset
# def preprocess_rte(batch):
#     encoding = tokenizer(
#         [f"{premise} Question: {hypothesis} Yes or No?" for premise, hypothesis in zip(batch["sentence1"], batch["sentence2"])],
#         truncation=True,
#         padding="max_length",
#         max_length=128
#     )
#     return encoding

def preprocess_rte(batch):
    encoding = tokenizer(
        [f'Given the statement "{sentence1}", does it necessarily follow that "{sentence2}" is true?'
         for sentence1, sentence2 in zip(batch["sentence1"], batch["sentence2"])],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors='pt'
    )
    batch["input_ids"] = encoding["input_ids"].squeeze()
    batch["attention_mask"] = encoding["attention_mask"].squeeze()
    return batch

# # preprocess hans
# def preprocess_hans(batch):
#     encoding = tokenizer(
#         [f"{premise} Question: {hypothesis} Yes or No?" for premise, hypothesis in zip(batch["premise"], batch["hypothesis"])],
#         truncation=True,
#         padding="max_length",
#         max_length=128
#     )
#     if "labels" not in batch:
#         encoding["labels"] = batch["label"]
#     return encoding

# # Ensure 'labels' column is correct for HANS
# if "labels" in hans_data["validation"].column_names:
#     print("'labels' column already exists in HANS. Skipping renaming.")
# else:
#     hans_data = hans_data.rename_column("label", "labels")

# # Remove redundant 'label' column
# if "label" in hans_data["validation"].column_names:
#     hans_data = hans_data.remove_columns(["label"])

# # Set format for PyTorch
# hans_data = hans_data.with_format("torch")

# # Preprocess HANS dataset
# def preprocess_hans(batch):
#     encoding = tokenizer(
#         [f"{premise} Question: {hypothesis} Yes or No?" for premise, hypothesis in zip(batch["premise"], batch["hypothesis"])],
#         truncation=True,
#         padding="max_length",
#         max_length=128
#     )
#     # Use 'labels' directly
#     encoding["labels"] = batch["labels"]
#     return encoding

def preprocess_hans(batch):
    fixed_context = "Given the premise, does the hypothesis hold true? "
    encoding = tokenizer(
        [f'{fixed_context} Premise: {premise} Hypothesis: {hypothesis}'
         for premise, hypothesis in zip(batch["premise"], batch["hypothesis"])],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors='pt'
    )
    batch["input_ids"] = encoding["input_ids"].squeeze()
    batch["attention_mask"] = encoding["attention_mask"].squeeze()
    return batch

# Load tokenizer and model config
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
config = AutoConfig.from_pretrained("facebook/opt-125m", num_labels=2, hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1)

# Preprocess datasets
rte_data = rte_data.map(preprocess_rte, batched=True)
hans_data = hans_data.map(preprocess_hans, batched=True)

# Inspect the first few rows of the HANS dataset
# print(hans_data["validation"].to_pandas().head())


# # Set dataset format to PyTorch
# rte_data = rte_data.rename_column("label", "labels").with_format("torch")
# hans_data = hans_data.rename_column("label", "labels").with_format("torch")

# Split datasets
train_dataset = rte_data["train"]
eval_dataset = rte_data["validation"]
hans_eval_dataset = hans_data["validation"]

# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Define parameters for training experiments (per reference paper)

few_shot_sample_size = [2, 32, 128]  # number of examples for each class
num_epochs = 40
batch_size = 32
learning_rate = 1e-5
weight_decay = 0.0
warmup_ratio = 0.1
num_runs = 3

# Vaiables to store training results into CSV
results_df = pd.DataFrame(columns=["n", "rank", "run", "in_domain_accuracy", "out_of_domain_accuracy"])

lora_configs = [
    LoraConfig(r=4, lora_alpha=8, lora_dropout=0.1, task_type="SEQ_CLS")
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/584k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/621k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

# **Experimentation loop**

In [7]:
import time
start_time = time.time()

for _, n in enumerate(tqdm(few_shot_sample_size)):
    for _, lora_config in enumerate(tqdm(lora_configs)):
      for run_idx in tqdm(range(num_runs)):  # repeat 10 times for each n
          # re-iniialize model for each run
          model = AutoModelForSequenceClassification.from_pretrained("facebook/opt-125m")
          model = get_peft_model(model, lora_config)
          optimizer = AdamW(model.parameters(), lr=learning_rate)

          # Select n random examples for each class from the original data
          indices_yes = [i for i, label in enumerate(train_dataset["label"]) if label == 0]
          indices_no = [i for i, label in enumerate(train_dataset["label"]) if label == 1]
          indices_yes = np.random.choice(indices_yes, n, replace=False)
          indices_no = np.random.choice(indices_no, n, replace=False)
          indices = np.concatenate([indices_yes, indices_no])

          # Select the examples for the new training set
          few_shot_train_dataset = train_dataset.select(indices)

          # Define training config
          # Total steps = (#samples/batch size) * epochs
          total_steps = (len(few_shot_train_dataset) // batch_size) * num_epochs

          training_args = TrainingArguments(
              output_dir = "./results",
              overwrite_output_dir = True,
              num_train_epochs = num_epochs,
              per_device_train_batch_size = batch_size,
              learning_rate = learning_rate,
              weight_decay = weight_decay,
              save_steps = 10_000,
              save_total_limit = 2,
              warmup_steps = int(warmup_ratio * total_steps),
          )

          # Define the trainer
          trainer = Trainer(
              model = model,
              args=training_args,
              train_dataset = train_dataset,
              compute_metrics = compute_metrics,
              optimizers=(optimizer, None),
          )

          # Train the model
          trainer.train()

          # Evaluate in-domain performance
          print(f"Evaluating in-domain performance for n={n}, rank={lora_config.r}...")
          eval_results_in_domain = trainer.evaluate(eval_dataset=eval_dataset)
          in_domain_accuracy = eval_results_in_domain["eval_accuracy"]

          # Print the in-domain evaluation results
          for key, value in eval_results_in_domain.items():
              print(f"In-domain {key}: {value}")

          # Evaluate out-of-domain performance
          print(f"Evaluating out-of-domain performance for n={n}, rank={lora_config.r}...")
          eval_results_out_domain = trainer.evaluate(eval_dataset=hans_eval_dataset)
          out_of_domain_accuracy = eval_results_out_domain["eval_accuracy"]

          # Print the out-of-domain evaluation results
          for key, value in eval_results_out_domain.items():
              print(f"Out-of-domain {key}: {value}")

          # Add the results to the DataFrame
          new_row = pd.DataFrame({
              "n": [n],
              "rank": [lora_config.r],
              "run": [run_idx],
              "in_domain_accuracy": [in_domain_accuracy],
              "out_of_domain_accuracy": [out_of_domain_accuracy]
          })
          results_df = pd.concat([results_df, new_row], ignore_index=True)

print(f'Finished training!!')
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

print(f"Time used: {elapsed_time:.2f} seconds")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-125m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Step,Training Loss
500,0.716400
1000,0.704600
1500,0.696700
2000,0.692200
2500,0.685800
3000,0.683500


Evaluating in-domain performance for n=128, rank=4...


In-domain eval_loss: 0.6895967125892639
In-domain eval_accuracy: 0.5740072202166066
In-domain eval_runtime: 2.0255
In-domain eval_samples_per_second: 136.758
In-domain eval_steps_per_second: 17.28
In-domain epoch: 40.0
Evaluating out-of-domain performance for n=128, rank=4...




100%|██████████| 3/3 [1:22:48<00:00, 1656.33s/it]

100%|██████████| 3/3 [4:09:23<00:00, 4987.81s/it]

Out-of-domain eval_loss: 0.7054479718208313
Out-of-domain eval_accuracy: 0.49106666666666665
Out-of-domain eval_runtime: 220.6365
Out-of-domain eval_samples_per_second: 135.97
Out-of-domain eval_steps_per_second: 16.996
Out-of-domain epoch: 40.0
Finished training!!
Time used: 14963.45 seconds


In [8]:
# Group the results by number of examples in few-shot learning runs and compute the average and maximum performance for each group
grouped_results = results_df.groupby(['n', 'rank'])

average_in_domain_accuracy = grouped_results['in_domain_accuracy'].mean()
maximum_in_domain_accuracy = grouped_results['in_domain_accuracy'].max()

average_out_of_domain_accuracy = grouped_results['out_of_domain_accuracy'].mean()
maximum_out_of_domain_accuracy = grouped_results['out_of_domain_accuracy'].max()

# Print the results for each 'n' and each rank
for n in few_shot_sample_size:
    for lora_config in lora_configs:
      print(f"For n={n}:")
      print(f"For rank={lora_config.r}:")
      print(f"Average in-domain accuracy: {average_in_domain_accuracy[n]}")
      print(f"Maximum in-domain accuracy: {maximum_in_domain_accuracy[n]}")
      print(f"Average out-of-domain accuracy: {average_out_of_domain_accuracy[n]}")
      print(f"Maximum out-of-domain accuracy: {maximum_out_of_domain_accuracy[n]}")
      print("\n")

# Compute and print the overall average and maximum performance
print("Overall:")
print(f"Average in-domain accuracy: {results_df['in_domain_accuracy'].mean()}")
print(f"Maximum in-domain accuracy: {results_df['in_domain_accuracy'].max()}")
print(f"Average out-of-domain accuracy: {results_df['out_of_domain_accuracy'].mean()}")
print(f"Maximum out-of-domain accuracy: {results_df['out_of_domain_accuracy'].max()}")


For n=2:
For rank=4:
Average in-domain accuracy: rank
4    0.570397
Name: in_domain_accuracy, dtype: float64
Maximum in-domain accuracy: rank
4    0.574007
Name: in_domain_accuracy, dtype: float64
Average out-of-domain accuracy: rank
4    0.4949
Name: out_of_domain_accuracy, dtype: float64
Maximum out-of-domain accuracy: rank
4    0.502433
Name: out_of_domain_accuracy, dtype: float64


For n=32:
For rank=4:
Average in-domain accuracy: rank
4    0.577617
Name: in_domain_accuracy, dtype: float64
Maximum in-domain accuracy: rank
4    0.577617
Name: in_domain_accuracy, dtype: float64
Average out-of-domain accuracy: rank
4    0.491533
Name: out_of_domain_accuracy, dtype: float64
Maximum out-of-domain accuracy: rank
4    0.491533
Name: out_of_domain_accuracy, dtype: float64


For n=128:
For rank=4:
Average in-domain accuracy: rank
4    0.574007
Name: in_domain_accuracy, dtype: float64
Maximum in-domain accuracy: rank
4    0.574007
Name: in_domain_accuracy, dtype: float64
Average out-of-domai

In [2]:
# Define the filename
curr_filename = "peft_Lora_RTE2.csv"

# Save the DataFrame to the current directory
results_df.to_csv(curr_filename, sep=',', index=False)
print(f"Results saved to {curr_filename} in the current directory.")



NameError: name 'results_df' is not defined

In [1]:
# Save the DataFrame to a CSV file
results_df.to_csv(f'../Results/{curr_filename.split(".")[0]}.csv', sep = ',', index=False)

NameError: name 'results_df' is not defined

In [ ]:
# # disconnect runtime
from google.colab import runtime
runtime.unassign()